In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("..")

In [2]:
from pathlib import Path

from librep.datasets.kuhar import (
    RawKuHar,
    RawKuHarIterator,
    KuHarDatasetGenerator,
    KuHarDataset,
)

In [3]:
dataset_dir = Path("../data/datasets/KuHar/1.Raw_time_domain_data")
kuhar_dataset = RawKuHar(dataset_dir, download=False)
kuhar_dataset

KuHar Dataset at: '../data/datasets/KuHar/1.Raw_time_domain_data'

In [4]:
iterator = RawKuHarIterator(kuhar_dataset)
iterator

Kuhar Iterator: users=89, activities=18

In [5]:
kuhar_v1 = KuHarDatasetGenerator(iterator, time_window=300, window_overlap=0)
kuhar_v1

Dataset generator: time_window=300, overlap=0, labels=class

In [6]:
train, validation, test = kuhar_v1.create_datasets(
    train_size=0.7,
    validation_size=0.1,
    test_size=0.2,
    ensure_distinct_users_per_dataset=True,
    balance_samples=True,
    seed=0
)

Generating full df over KuHar View: 1945it [01:30, 21.51it/s]


TypeError: train_test_split() got an unexpected keyword argument 'seed'

In [ ]:
train_dataset_accel = KuHarDataset(train, sensors=["accel-x", "accel-y", "accel-z"], label_columns=["class", "user"], as_array=False)
train_dataset = KuHarDataset(train)
validation_dataset = KuHarDataset(validation)
test_dataset = KuHarDataset(test)


print(train_dataset_accel)
print(train_dataset)
print(validation_dataset)
print(test_dataset)

In [ ]:
from librep.utils.dataset import load_full_data

x, y = load_full_data(train_dataset, return_X_y=True)

In [ ]:
x.shape, y.shape

In [ ]:
x[10], y[10]